In [3]:
from transformers import AutoProcessor, MusicgenForConditionalGeneration

processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")

# inputs = processor(
#     text=["80s pop track with bassy drums and synth", "90s rock song with loud guitars and heavy drums"],
#     padding=True,
#     return_tensors="pt",
# )
# audio_values = model.generate(**inputs, do_sample=True, guidance_scale=3, max_new_tokens=256)

/Users/michaelxue/anaconda3/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [2]:
print(model)

MusicgenForConditionalGeneration(
  (text_encoder): T5EncoderModel(
    (shared): Embedding(32128, 768)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=768, out_features=768, bias=False)
                (k): Linear(in_features=768, out_features=768, bias=False)
                (v): Linear(in_features=768, out_features=768, bias=False)
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=768, out_features=3072, bias=False)
                (wo): L

In [3]:
for name, param in model.named_parameters():
    print(f"Layer: {name}, Requires Gradient: {param.requires_grad}")


Layer: text_encoder.shared.weight, Requires Gradient: True
Layer: text_encoder.encoder.block.0.layer.0.SelfAttention.q.weight, Requires Gradient: True
Layer: text_encoder.encoder.block.0.layer.0.SelfAttention.k.weight, Requires Gradient: True
Layer: text_encoder.encoder.block.0.layer.0.SelfAttention.v.weight, Requires Gradient: True
Layer: text_encoder.encoder.block.0.layer.0.SelfAttention.o.weight, Requires Gradient: True
Layer: text_encoder.encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight, Requires Gradient: True
Layer: text_encoder.encoder.block.0.layer.0.layer_norm.weight, Requires Gradient: True
Layer: text_encoder.encoder.block.0.layer.1.DenseReluDense.wi.weight, Requires Gradient: True
Layer: text_encoder.encoder.block.0.layer.1.DenseReluDense.wo.weight, Requires Gradient: True
Layer: text_encoder.encoder.block.0.layer.1.layer_norm.weight, Requires Gradient: True
Layer: text_encoder.encoder.block.1.layer.0.SelfAttention.q.weight, Requires Gradient: True
Layer

In [4]:
from peft import LoraModel, LoraConfig, get_peft_model
config = LoraConfig(r=16, target_modules=['k_proj', 'q_proj', 'v_proj', 'out_proj'], lora_alpha=32, lora_dropout=0.01)

lora_model = get_peft_model(model, config)
# when doing this with the actual code in the music gen solver class, let's also check requires_grad before training/finetuning starts